In [ ]:
import re
import os
import warnings
import random
warnings.filterwarnings('ignore')
from datasets import Dataset, DatasetDict
from torch.utils.data import Dataset as TorchDataset

### 데이터 로드

In [1]:
train_texts = []
with open("/home/elicer/WKH/data/train_data.txt", "r") as file:
    for i in file:
        train_texts.append(i.strip())

len(train_texts)

1930

### 슬라이딩 윈도우 설정

In [5]:
import torch
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/elicer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
# 슬라이딩 윈도우 파라미터
window_size = 10  # 윈도우 크기 (문장 수)
sliding_step = 5  # 슬라이딩 간격 (문장 수)

def sliding_window(sentences, window_size, step):
    """ 문장 리스트에 슬라이딩 윈도우를 적용하는 함수 """
    windows = []
    for start in range(0, len(sentences) - window_size + 1, step):
        window = sentences[start:start + window_size]
        windows.append(window)
    
    # 마지막 윈도우 추가 (중복 방지)
    if len(sentences) % step != 0:
        last_window = sentences[-window_size:]
        if last_window not in windows:
            windows.append(last_window)
    return windows


# 동화 데이터에 슬라이딩 윈도우 적용
all_story_windows = []

for story in train_texts:
    # 동화에서 문장 분리
    sentences = sent_tokenize(story)

    # 슬라이딩 윈도우 적용
    story_windows = sliding_window(sentences, window_size, sliding_step)
    
    all_story_windows.append(story_windows)

In [ ]:
# 예를 들어 문장 수가 충분하지 않은 경우를 확인
for i, story in enumerate(train_texts):
    sentences = sent_tokenize(story)
    if len(sentences) < window_size:
        print(f"Story {i} has less than {window_size} sentences and cannot have any windows.")

In [13]:
# 결과를 저장할 리스트
all_new_story = []

# 전체 all_story_windows에 대해 반복
for story_window in all_story_windows:
    # 문장들을 저장할 리스트
    sentences_list = []
    
    for sentences in story_window:
        # 각 문장을 sentences_list에 추가
        sentences_list.extend(sentences)
    
    # 문장들을 하나의 문자열로 결합 (문장 사이에만 공백을 추가)
    new_story = ' '.join(sentences_list)

    # 줄바꿈 문자 제거 및 앞뒤 공백 제거
    new_story = new_story.replace('\n', ' ').strip()
    
    # 결과 리스트에 추가
    all_new_story.append(new_story)

In [ ]:
print(len(all_new_story))
all_new_story[397]

In [15]:
train_texts = all_new_story

In [ ]:
import torch
import numpy as np
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel, PreTrainedTokenizerFast, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader, SubsetRandomSampler
import wandb

wandb.login()

# GPU 환경에서 사용 가능하도록 변경
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# SKT에서 개발한 한국어 GPT-2 모델, (한국어 텍스트의 생성, 분류, 번역) 등 다양한 자연어 처리 작업에 사용할 수 있는 사전 훈련된 모델
model_name = 'skt/kogpt2-base-v2'
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)


# 토크나이저 로드  (토크나이저는 텍스트를 입력으로 받아서 모델이 처리할 수 있는 형식으로 변환하고, 반대로 모델의 출력을 해석할 수 있는 역할)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

In [18]:
# 해당 토큰들은 추가적인 토큰으로, 추가하거나 빼거나 하시면 됩니다!
tokenizer.add_special_tokens({'pad_token': '[PAD]'})   # 패딩 토큰 (일정한 길이로 맞추기 위해 사용)
tokenizer.add_special_tokens({'bos_token': '<BOS>'})   # 시작 토큰 (더욱더 동화스럽게 만들기 위해 시작 구문 추가) - ex) 옛날 옛날에~
tokenizer.add_special_tokens({'eos_token': '<EOS>'})   # 종료 토큰 (더욱더 동화스럽게 만들기 위해 끝 맺음 추가) - ex) 행복하게 살았답니다~
tokenizer.add_special_tokens({'sep_token': '<SEP>'})   # 문장 경계 토큰 (새로운 장면이나 시간이 흐른 것을 알리는 문구)

# 모델의 임베딩 테이블 크기를 토크나이저 설정에 맞게 재조정
model.resize_token_embeddings(len(tokenizer))

# 꿈틀 데이터셋 클래스 정의
class DreamTwistDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=max_length)

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# 데이터셋 인스턴스 생성
max_length = 512  # 문장 최대 길이로 설정해주시면 됩니다.

# 꿈틀 트레인 데이터셋 설정
train_dataset = DreamTwistDataset(train_texts, tokenizer, max_length)

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=max_length
)

In [14]:
# 트레이닝 인자 설정 (여러분들이 인공지능 수업에서 배운 노하우를 활용하여 에포크, 배치사이즈, 스탭 등 자유롭게 조절하여 학습 인자 셋팅)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=50 ,
    per_device_train_batch_size=1,  # 배치 사이즈 조정
    save_steps=10_000,
    save_total_limit=2,
    eval_strategy='epoch',
    logging_dir='./logs',
    logging_steps=100,
    logging_first_step=True,
    learning_rate=5e-5,
    overwrite_output_dir=True,
)

# 트레이너 객체 생성 및 훈련
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    data_collator=data_collator,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: wkh1204 (wkh1204-elice). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.963000,1.608692
2,1.787800,1.338791
3,1.507600,1.095997
4,1.320600,0.862129
5,1.071700,0.658608
6,0.920100,0.485693
7,0.720600,0.349013
8,0.569800,0.234010
9,0.480800,0.157418
10,0.343400,0.105507


In [ ]:
output_dir = './trained_model/240727_KH_data1930total_newWindows10-5_epoch50_batchSize1'
# 모델 저장
model.save_pretrained(output_dir)
# 토크나이저 저장
tokenizer.save_pretrained(output_dir)

In [407]:
# 동화 생성 함수

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def make_fairy_tale_story(prompt, model, tokenizer, max_length=512, min_length=100, num_beams=5, temperature=0.7, top_k=50, top_p=0.95):
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    attention_mask = torch.ones_like(input_ids).to(device)
    num_return_sequences = 1

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids.to(model.device),
            attention_mask=attention_mask.to(model.device),
            max_length=max_length,
            min_length=min_length,
            num_return_sequences=num_return_sequences,
            no_repeat_ngram_size=4,
            num_beams = num_beams,
            do_sample=True,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

In [ ]:
# 동화 생성
prompt = '아주 먼 옛날 왕자가 살고 있었어요.'
fairy_tale_story = make_fairy_tale_story(prompt, model, tokenizer, max_length=512)
print(fairy_tale_story)

In [378]:
# 모델 학습 후 GPU 메모리 초기화
def clear_gpu_memory():
    # GPU 메모리의 캐시 비우기
    torch.cuda.empty_cache()

    # GPU 메모리 사용량 확인 (선택 사항)
    print("GPU 메모리 사용량:")
    print(torch.cuda.memory_summary())

# 모델 학습 완료 후 GPU 메모리 초기화
clear_gpu_memory()

GPU 메모리 사용량:
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1472 MiB |   4313 MiB | 131346 GiB | 131344 GiB |
|       from large pool |   1459 MiB |   4300 MiB | 129666 GiB | 129664 GiB |
|       from small pool |     13 MiB |     87 MiB |   1679 GiB |   1679 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   1472 MiB |   4313 MiB | 131346 GiB | 131344 GiB |
|       from large pool |   1459 MiB |   4300 MiB |